In [137]:
AZURE_OPENAI_ENDPOINT = ""
API_KEY = ""

In [132]:
import requests
import json

JSON_INDENT = 2
def format_json(obj):
    """
    Formats a given JSON instance into a readable string with a defined indentation.

    Args:
        obj (Any): The JSON object to format, which should be
            a valid Python dictionary or list structure.

    Returns:
        str: A JSON-formatted string with indentation specified by the global JSON_INDENT.
    """
    try:
        return json.dumps(obj, indent=JSON_INDENT)
    except Exception as e:# (TypeError, ValueError)
        print(f"Error formatting JSON: {e}")

def create_response_format(schema):
    response_format = {
        "type": "json_schema",
        "json_schema": {
            "name": "json_output_schema",
            "schema": {**schema},
            "strict": True
        }
    }
    return response_format

# Replace with your Azure OpenAI details

API_VERSION = "2024-08-01-preview"
DEPLOYMENT_NAME = "gpt-4o-strict-json"

# Endpoint for creating an assistant
url = f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"

headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY
}

In [133]:
classification_types = ["world_news", "finance", "sports", "politics", "local_news"]
json_output_schema = {
    "type": "object",
    "properties": {
        "response": {
            "type": "object",
            "properties": {
                    "reasoning": { "type": "string" },
                    "classification": { "type": "string", "enum": classification_types}
            },
            "additionalProperties": False,
            "required": ["reasoning", "classification"]
        }
    },
    "additionalProperties": False,
    "required": ["response"]
}
response_format = create_response_format(json_output_schema)

json_output_instance = {
    "response": {
            "reasoning": "This news article is of the type internal_news because of bla bla bla",
            "classification": "internal_news"
    }
}

In [134]:
system_prompt = f"""You are a journalist with over 25 years of experience with great expertise in reading and classifying news articles.
You will receive the content of a news article and your task is to classify it into a set of pre-defined news types.
Please read the entire article in full and then respond in the following format:
{format_json(json_output_instance)}
Add your reasoning to the "reasoning" entry.
Add the classification to the "classification" entry.
"""

In [135]:
user_prompt = """
"""

In [136]:
data = {
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    "response_format": response_format
}
response = json.loads(requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"])["response"]
response

{'reasoning': 'This news article is of the type local_news because it focuses on events or stories happening within a specific community or region. The content likely involves issues, incidents, or developments that primarily affect a local audience and is structured around a geographical or communal focus.',
 'classification': 'local_news'}